<a href="https://colab.research.google.com/github/danykoud/Test-/blob/main/Natural_language_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Do not change anything in this cell
import re
import string
from os import getcwd

import pdb

import numpy as np
import pandas as pd
from tkinter import *
import nltk
from nltk.corpus import stopwords, twitter_samples
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# TASK 1 CELL

# get the sets of positive and negative tweets
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
# for i in positive_tweets:
#   print(i)
for j in positive_tweets:
  print(j)

LookupError: ignored

In [ ]:
label_positive = [1] * len(positive_tweets)
label_negative = [0] * len(negative_tweets)
print(label_positive)
print(label_negative)

In [ ]:
# TASK 1 CELL

# get the sets of positive and negative tweets
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

label_positive = [1] * len(positive_tweets)
label_negative = [0] * len(negative_tweets)

labels = label_positive + label_negative
tweets = positive_tweets + negative_tweets

# split the data into train and validation set

train_x, test_x, train_y, test_y = train_test_split(tweets, labels, train_size=0.8)

In [ ]:
training_1 = pd.DataFrame({'tweets':train_x, 'labels':train_y})
training_1

In [ ]:
training = pd.DataFrame.from_dict(dict(tweet=train_x, label=train_y))
training

In [ ]:
sample = training[(training.tweet.str.contains("beautiful")) &
                      (training.tweet.str.contains("http")) &
                      (training.tweet.str.contains("#"))]
samp= sample.iloc[0].tweet
samp


In [ ]:
# TASK 1 CELL
#remove hyperlinks, noise, punctuation, hashtags, old style retweet
#text "RT", share market tickers like $FB, use stemming to only keep
#track of one variation of each word and in the end will tokenize them. 
# result = re.sub(pattern, repl, string, count=0, flags=0);
# result = re.sub('abc',  '',    input)           # Delete pattern abc
# result = re.sub('abc',  'def', input)           # Replace pattern abc -> def
# result = re.sub(r'\s+', ' ',   input)           # Eliminate duplicate whitespaces using wildcards
# result = re.sub('abc(def)ghi', r'\1', input)    # Replace a string with a part of itself

def clean_tweet(tweet):
    '''
    Params:
        tweet: a string containing a tweet
    Return:
        tweets_clean: a list of words containing the processed tweet

    '''

    # it will remove the old style retweet text "RT"
    tweet2 = re.sub(r'^RT[\s]+', '', tweet)

    # it will remove hyperlinks
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)

    # it will remove hashtags. We have to be careful here not to remove 
    # the whole hashtag because text of hashtags contains huge information. 
    # only removing the hash # sign from the word
    tweet2 = re.sub(r'#', '', tweet2)

    # it will remove single numeric terms in the tweet. 
    tweet2 = re.sub(r'[0-9]', '', tweet2)
    #print('\nAfter removing old style tweet, hyperlinks and # sign')
    #print(tweet2)
    
    #tweet2 = re.sub(r"\$"+"{}*",format(.),'')
    tweet2 = re.sub(r'\$[\w,]+\b','', tweet2)

    # instantiate the tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, 
                               strip_handles=True,
                               reduce_len=True)

    # tokenize the tweets 
    tweet_tokens = tokenizer.tokenize(tweet2)

    #print('\nTokenized string:')
    #print(tweet_tokens)
    
    #Import the english stop words list from NLTK
    stopwords_english = stopwords.words('english') 
    
    tweets_clean = []

    for word in tweet_tokens: # Go through every word in your tokens list
        if (word not in stopwords_english and 
            word not in string.punctuation):  
            tweets_clean.append(word)

    #print('\n\nAfter removing stop words and punctuation:')
    #print(tweets_clean)
    
    # Instantiate stemming class
    stemmer = PorterStemmer() 

    # Create an empty list to store the stems
    tweets_stem = [] 

    for word in tweets_clean:
        stem_word = stemmer.stem(word)  # stemming word
        tweets_stem.append(stem_word)  # append to the list

    #print('Words after stemming: ')
    # print(tweets_stem)
    
    return tweets_stem

In [ ]:

clean_tweet(samp)

In [ ]:
# TASK 1 CELL

# Helper function to find word frequency
def find(frequency, word, label):
    '''
    Params:
        frequency: a dictionary with the frequency of each pair (or tuple)
        word: the word to look up
        label: the label corresponding to the word
    Return:
        n: the number of times the word with its corresponding label appears.
    '''
    n = 0  
    
    word_pair = (label, word)
    print(word_pair)
    #print(frequency.keys)
    print(frequency)
    #print(frequency[word_pair])
    
    
    for word_pair in frequency.items():
        
        #n = frequency[word_pair]
        n += 1
        print('made it')
        print(n)
    
    print(n)
    return n

In [ ]:
# TASK 2 CELL

def tweet_counter(output, tweets, tweet_senti):
    '''
    Params:
        output: a dictionary that will be used to map each pair to its frequency
        tweets: a list of tweets
        tweet_senti: a list corresponding to the sentiment of each tweet (either 0 or 1)
    Return:
        output: a dictionary mapping each pair to its frequency
    '''
 
    for label, tweet in zip(tweet_senti, tweets):
        for word in clean_tweet(tweet):
            if (word, label) in output.keys():
                output[(word, label)] = output[(word, label)] + 1
                
            else: 
                output[(word, label)] = 1
   
    return output

frequency_dict = tweet_counter({}, tweets, labels)
frequency_dict_pos = tweet_counter({}, tweets, label_positive)
frequency_dict_neg = tweet_counter({}, tweets, label_negative)

In [ ]:
frequency_dict

In [ ]:

freqs= frequency_dict

In [ ]:
# TASK 3 CELL

def train_naive_bayes(frequency_dict, train_x, train_y):
    '''
    Params:
        frequency_dict: dictionary from (word, label) to how often the word appears
        train_x: a list of tweets
        train_y: a list of labels correponding to the tweets (0,1)
    Return:
        logprior: the log prior. (equation 3 above)
        loglikelihood: the log likelihood of you Naive bayes equation. (equation 6 above)
    '''
    loglikelihood = {}
    class_priors={}
    logprior = 0


    # calculate V, the number of unique words in the vocabulary
    vocab = frequency_dict.items()
    V = len(vocab)

    # calculate num_pos and num_neg - the total number of positive and negative words for all documents
    num_pos = num_neg = 0
    for pair in frequency_dict.keys():
        # if the label is positive (greater than zero)
      for n in labels:
        if n > 0:

            # Increment the number of positive words by the count for this (word, label) pair
            num_pos += 1

        # else, the label is negative
        else:

            # increment the number of negative words by the count for this (word,label) pair
            num_neg += 1

    # Calculate num_doc, the number of documents
    
    num_doc =train_size= len(train_y)
    # N_pos is number of positive documents
    N_pos = np.sum(train_y)
    # N_neg is the number of negative documents
    N_neg = num_doc - N_pos
    
     # the log odds ratio
    logprior = np.log(N_pos) - np.log(N_neg)
    

    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = len(frequency_dict_pos)
        freq_neg = len(frequency_dict_neg)
        # calculate the probability that each word is positive, and negative
        p_w_pos =  (freq_pos + 1)/(num_pos+ V)
        p_w_neg = (freq_neg + 1)/(num_neg + V)

        # calculate the log likelihood of the word
        # need to check the equation/------->loglikelihood: the log likelihood of you Naive bayes equation. (equation 6 above)
        loglikelihood[word] =  np.log(p_w_pos) - np.log(p_w_neg)


    return logprior, loglikelihood

In [ ]:
# Do not change anything in this cell
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

In [ ]:
# TASK 4 CELL

def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    Params:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Return:
        total_prob: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)

    '''
    # process the tweet to get a list of words
    word_l = [words for words in tweet]

    # initialize probability to zero
    total_prob = 0

    # add the logprior
    total_prob =logprior + sum(loglikelihood[word] for word in word_l if word in loglikelihood)



    return total_prob

In [ ]:
# Do not change anything in this cell
custom_tweet = 'I love NLP'
prob = naive_bayes_predict(custom_tweet, logprior, loglikelihood)
print('The expected output is', prob)

In [ ]:
# TASK 5 CELL

def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Params:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Return:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0  

    
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if None:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats

    # error is the average of the absolute values of the differences between y_hats and test_y
    # TASK 5 CELL

def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Params:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Return:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0  

    
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if None:
            # the predicted class is 1
            y_hat_i = None
        else:
            # otherwise the predicted class is 0
            y_hat_i = None

        # append the predicted class to the list y_hats
        y_hats = None

    # error is the average of the absolute values of the differences between y_hats and test_y
    # TASK 5 CELL

def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Params:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Return:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0  

    
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) < 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    
    error = np.mean(np.abs(np.array(y_hats) - np.array(test_y)))
    
    accuracy = 1 - error 


    return accuracy

In [ ]:
# Do not change anything in this cell
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))